### 데이터셋 생성

In [9]:
import pandas as pd

In [ ]:
# Matches.csv: Elo, 배당률, Form, 기본 스탯 포함된 Kaggle 원본
matches = pd.read_csv("Matches.csv")

xg = pd.read_csv("xg_data.csv")

# 날짜 전처리
matches['MatchDate'] = pd.to_datetime(matches['MatchDate'], errors='coerce')
xg['date'] = pd.to_datetime(xg['date'], errors='coerce')


C:\Users\haeun\AppData\Local\Temp\ipykernel_6976\3155053130.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  matches = pd.read_csv("Matches.csv")


In [11]:
matches['HomeTeam'] = matches['HomeTeam'].str.strip()
matches['AwayTeam'] = matches['AwayTeam'].str.strip()
xg['team_h'] = xg['team_h'].str.strip()
xg['team_a'] = xg['team_a'].str.strip()

matches['merge_key'] = matches['MatchDate'].astype(str) + '_' + matches['HomeTeam'] + '_' + matches['AwayTeam']
xg['merge_key'] = xg['date'].astype(str) + '_' + xg['team_h'] + '_' + xg['team_a']


In [12]:
merged = matches.merge(
    xg[['merge_key', 'h_xg', 'a_xg', 'xG_diff']],
    on='merge_key',
    how='left'
)

In [13]:
# 결과 레이블
def encode_result(r):
    if r == 'H':
        return 2
    elif r == 'D':
        return 1
    elif r == 'A':
        return 0
    return None

merged['result'] = merged['FTResult'].apply(encode_result)

# Elo 차이
merged['elo_diff'] = merged['HomeElo'] - merged['AwayElo']

# Form 처리 (결측값 -> 0)
for col in ['Form3Home', 'Form5Home', 'Form3Away', 'Form5Away']:
    merged[col] = merged[col].fillna(0)

# 경기 스탯 차이
merged['shots_diff'] = merged['HomeShots'] - merged['AwayShots']
merged['target_diff'] = merged['HomeTarget'] - merged['AwayTarget']
merged['fouls_diff'] = merged['HomeFouls'] - merged['AwayFouls']
merged['corners_diff'] = merged['HomeCorners'] - merged['AwayCorners']
merged['yellow_diff'] = merged['HomeYellow'] - merged['AwayYellow']
merged['red_diff'] = merged['HomeRed'] - merged['AwayRed']

# 배당률 → 확률화
merged['prob_home'] = 1 / merged['OddHome']
merged['prob_draw'] = 1 / merged['OddDraw']
merged['prob_away'] = 1 / merged['OddAway']
prob_sum = merged['prob_home'] + merged['prob_draw'] + merged['prob_away']
merged['prob_home'] /= prob_sum
merged['prob_draw'] /= prob_sum
merged['prob_away'] /= prob_sum

# 시간 피처
merged['month'] = pd.to_datetime(merged['MatchDate']).dt.month
merged['weekday'] = pd.to_datetime(merged['MatchDate']).dt.weekday

# xG 관련 추가 피처
merged['xg_margin'] = merged['h_xg'] - merged['a_xg']
merged['xg_ratio'] = merged['h_xg'] / (merged['a_xg'] + 1e-6)  # ZeroDivision 방지


In [15]:
cols = [
    'HomeElo', 'AwayElo', 'elo_diff',
    'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away',
    'shots_diff', 'target_diff', 'fouls_diff', 'corners_diff',
    'yellow_diff', 'red_diff',
    'prob_home', 'prob_draw', 'prob_away',
    'h_xg', 'a_xg', 'xG_diff', 'xg_margin', 'xg_ratio',
    'month', 'weekday', 'result'
]

final_df = merged[cols].dropna(subset=['result'])  # 결과 없는 경기 제외

print("✅ 통합 데이터셋 shape:", final_df.shape)
final_df.head()

✅ 통합 데이터셋 shape: (230554, 24)


,HomeElo,AwayElo,elo_diff,Form3Home,Form5Home,Form3Away,Form5Away,shots_diff,target_diff,fouls_diff,...,prob_draw,prob_away,h_xg,a_xg,xG_diff,xg_margin,xg_ratio,month,weekday,result
0,1686.34,1586.57,99.77,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,0.265432,0.203704,NaN,NaN,NaN,NaN,NaN,7,4,2.0
1,1714.89,1642.51,72.38,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,0.258790,0.191280,NaN,NaN,NaN,NaN,NaN,7,4,2.0
2,1465.08,1633.80,-168.72,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,4,0.0
3,1635.58,1624.22,11.36,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,0.264038,0.223417,NaN,NaN,NaN,NaN,NaN,7,5,0.0
4,1734.34,1673.11,61.23,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,0.265432,0.203704,NaN,NaN,NaN,NaN,NaN,7,5,1.0


In [16]:
final_df.to_csv("merged_dataset.csv", index=False)

---

In [23]:
import pandas as pd

# 1. 원본 데이터 불러오기 --------------------------------------------------

# Matches.csv (Kaggle 제공 데이터)
matches = pd.read_csv("Matches.csv")

# xG 데이터 (예: understat 등에서 수집한 경기당 xG 기록)
xg = pd.read_csv("xg_data.csv")

# 날짜 처리
matches['MatchDate'] = pd.to_datetime(matches['MatchDate'], errors='coerce').dt.date
xg['date'] = pd.to_datetime(xg['date'], errors='coerce').dt.date

# 공백 제거 (팀 이름)
matches['HomeTeam'] = matches['HomeTeam'].str.strip()
matches['AwayTeam'] = matches['AwayTeam'].str.strip()
xg['team_h'] = xg['team_h'].str.strip()
xg['team_a'] = xg['team_a'].str.strip()

# 2. 병합 키 생성 ---------------------------------------------------------

matches['merge_key'] = matches['MatchDate'].astype(str) + '_' + matches['HomeTeam'] + '_' + matches['AwayTeam']
xg['merge_key'] = xg['date'].astype(str) + '_' + xg['team_h'] + '_' + xg['team_a']

# 3. xG 데이터 병합 -------------------------------------------------------

merged = matches.merge(
    xg[['merge_key', 'h_xg', 'a_xg']],  # 또는 xG_diff 포함 시 'xG_diff'
    on='merge_key',
    how='left'
)

# 4. 예측용 피처 정리 -----------------------------------------------------

# 결과 라벨 인코딩 (FTResult: H/ D / A → 2 / 1 / 0)
def encode_result(r):
    if r == 'H':
        return 2
    elif r == 'D':
        return 1
    elif r == 'A':
        return 0
    return None

merged['result'] = merged['FTResult'].apply(encode_result)

# Elo 차이
merged['elo_diff'] = merged['HomeElo'] - merged['AwayElo']

# 배당률을 확률로 환산
merged['prob_home'] = 1 / merged['OddHome']
merged['prob_draw'] = 1 / merged['OddDraw']
merged['prob_away'] = 1 / merged['OddAway']
prob_sum = merged['prob_home'] + merged['prob_draw'] + merged['prob_away']
merged['prob_home'] /= prob_sum
merged['prob_draw'] /= prob_sum
merged['prob_away'] /= prob_sum

# 날짜 기반 피처
merged['month'] = pd.to_datetime(merged['MatchDate']).dt.month
merged['weekday'] = pd.to_datetime(merged['MatchDate']).dt.weekday

# xG 관련 추가 피처
merged['xG_diff'] = merged['h_xg'] - merged['a_xg']
merged['xg_margin'] = merged['xG_diff']  # 같은 값
merged['xg_ratio'] = merged['h_xg'] / (merged['a_xg'] + 1e-6)

# 5. 필요한 컬럼만 추출 ---------------------------------------------------

selected_cols = [
    'MatchDate', 'HomeTeam', 'AwayTeam',
    'HomeElo', 'AwayElo', 'elo_diff',
    'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away',
    'prob_home', 'prob_draw', 'prob_away',
    'h_xg', 'a_xg', 'xG_diff', 'xg_margin', 'xg_ratio',
    'month', 'weekday', 'result'
]

# 존재하는 컬럼만 선택
final_cols = [col for col in selected_cols if col in merged.columns]
final_df = merged[final_cols].copy()

# 6. 저장 ------------------------------------------------------------------

final_df.to_csv("merged_dataset.csv", index=False)
print("✅ 통합 완료: merged_dataset.csv 생성됨")


C:\Users\haeun\AppData\Local\Temp\ipykernel_6976\1126825244.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  matches = pd.read_csv("Matches.csv")


✅ 통합 완료: merged_dataset.csv 생성됨


In [24]:
import pandas as pd

# 1. 데이터 로딩
df = pd.read_csv("merged_dataset.csv")
df['MatchDate'] = pd.to_datetime(df['MatchDate'])

# 2. 홈/원정 경기 데이터 결합
home = df[['MatchDate', 'HomeTeam', 'h_xg', 'HomeElo']].copy()
home.columns = ['date', 'team', 'xg', 'elo']

away = df[['MatchDate', 'AwayTeam', 'a_xg', 'AwayElo']].copy()
away.columns = ['date', 'team', 'xg', 'elo']

team_games = pd.concat([home, away], axis=0).sort_values(['team', 'date']).reset_index(drop=True)

# 3. 롤링 피처 계산
team_games['rolling_xg_5'] = team_games.groupby('team')['xg'].transform(lambda x: x.shift().rolling(5, min_periods=1).mean())
team_games['elo_change'] = team_games.groupby('team')['elo'].transform(lambda x: x.diff())

# 4. 원본 df에 매핑 (홈팀)
df = df.merge(
    team_games[['team', 'date', 'rolling_xg_5', 'elo_change']],
    how='left',
    left_on=['HomeTeam', 'MatchDate'],
    right_on=['team', 'date']
)
df = df.rename(columns={
    'rolling_xg_5': 'rolling_xg_home_5',
    'elo_change': 'elo_change_home'
})
df = df.drop(columns=['team', 'date'])

# 5. 원본 df에 매핑 (원정팀)
df = df.merge(
    team_games[['team', 'date', 'rolling_xg_5', 'elo_change']],
    how='left',
    left_on=['AwayTeam', 'MatchDate'],
    right_on=['team', 'date']
)
df = df.rename(columns={
    'rolling_xg_5': 'rolling_xg_away_5',
    'elo_change': 'elo_change_away'
})
df = df.drop(columns=['team', 'date'])

# 6. 결과 확인
print(df[['HomeTeam', 'AwayTeam', 'MatchDate', 'rolling_xg_home_5', 'rolling_xg_away_5', 'elo_change_home', 'elo_change_away']].head())

# 7. 저장
df.to_csv("enhanced_dataset.csv", index=False)
print("✅ 저장 완료: enhanced_dataset.csv")


    HomeTeam    AwayTeam  MatchDate  rolling_xg_home_5  rolling_xg_away_5  \
0  Marseille      Troyes 2000-07-28                NaN                NaN   
1   Paris SG  Strasbourg 2000-07-28                NaN                NaN   
2  Wasquehal       Nancy 2000-07-28                NaN                NaN   
3    Auxerre       Sedan 2000-07-29                NaN                NaN   
4   Bordeaux        Metz 2000-07-29                NaN                NaN   

   elo_change_home  elo_change_away  
0              NaN              NaN  
1              NaN              NaN  
2              NaN              NaN  
3              NaN              NaN  
4              NaN              NaN  
✅ 저장 완료: enhanced_dataset.csv


---

In [ ]:
import pandas as pd

# 1. 원본 데이터 불러오기 --------------------------------------------------

# Matches.csv (Kaggle 제공 데이터)
matches = pd.read_csv("Matches.csv")

# xG 데이터 (예: understat 등에서 수집한 경기당 xG 기록)
xg = pd.read_csv("xg_data.csv")

# 날짜 처리
matches['MatchDate'] = pd.to_datetime(matches['MatchDate'], errors='coerce').dt.date
xg['date'] = pd.to_datetime(xg['date'], errors='coerce').dt.date

# 공백 제거 (팀 이름)
matches['HomeTeam'] = matches['HomeTeam'].str.strip()
matches['AwayTeam'] = matches['AwayTeam'].str.strip()
xg['team_h'] = xg['team_h'].str.strip()
xg['team_a'] = xg['team_a'].str.strip()

# 2. 병합 키 생성 ---------------------------------------------------------

matches['merge_key'] = matches['MatchDate'].astype(str) + '_' + matches['HomeTeam'] + '_' + matches['AwayTeam']
xg['merge_key'] = xg['date'].astype(str) + '_' + xg['team_h'] + '_' + xg['team_a']

# 3. xG 데이터 병합 -------------------------------------------------------

merged = matches.merge(
    xg[['merge_key', 'h_xg', 'a_xg']],  # 또는 xG_diff 포함 시 'xG_diff'
    on='merge_key',
    how='left'
)

# 4. 예측용 피처 정리 -----------------------------------------------------

# 결과 라벨 인코딩 (FTResult: H/ D / A → 2 / 1 / 0)
def encode_result(r):
    if r == 'H':
        return 2
    elif r == 'D':
        return 1
    elif r == 'A':
        return 0
    return None

merged['result'] = merged['FTResult'].apply(encode_result)

# Elo 차이
merged['elo_diff'] = merged['HomeElo'] - merged['AwayElo']

# 배당률을 확률로 환산
merged['prob_home'] = 1 / merged['OddHome']
merged['prob_draw'] = 1 / merged['OddDraw']
merged['prob_away'] = 1 / merged['OddAway']
prob_sum = merged['prob_home'] + merged['prob_draw'] + merged['prob_away']
merged['prob_home'] /= prob_sum
merged['prob_draw'] /= prob_sum
merged['prob_away'] /= prob_sum

# 날짜 기반 피처
merged['month'] = pd.to_datetime(merged['MatchDate']).dt.month
merged['weekday'] = pd.to_datetime(merged['MatchDate']).dt.weekday

# xG 관련 추가 피처
merged['xG_diff'] = merged['h_xg'] - merged['a_xg']
merged['xg_margin'] = merged['xG_diff']  # 같은 값
merged['xg_ratio'] = merged['h_xg'] / (merged['a_xg'] + 1e-6)

# 5. 필요한 컬럼만 추출 ---------------------------------------------------

selected_cols = [
    'MatchDate', 'HomeTeam', 'AwayTeam',
    'HomeElo', 'AwayElo', 'elo_diff',
    'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away',
    'prob_home', 'prob_draw', 'prob_away',
    'h_xg', 'a_xg', 'xG_diff', 'xg_margin', 'xg_ratio',
    'month', 'weekday', 'result'
]

# 존재하는 컬럼만 선택
final_cols = [col for col in selected_cols if col in merged.columns]
final_df = merged[final_cols].copy()


# 6. 저장 ------------------------------------------------------------------

final_df.to_csv("merged_dataset.csv", index=False)
print("✅ 통합 완료: merged_dataset.csv 생성됨")
